# Abstract Model

In [ ]:
import pyomo.environ as pyo

In [2]:
model = pyo.AbstractModel()

In [3]:
# Build the index set
model.Crop = pyo.Set()

In [4]:
# Build parameters
model.w_labor = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.w_capital = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.profit = pyo.Param(model.Crop,within=pyo.NonNegativeReals)
model.avail_land = pyo.Param(within=pyo.NonNegativeReals)
model.avail_labor = pyo.Param(within=pyo.NonNegativeReals)
model.avail_capital = pyo.Param(within=pyo.NonNegativeReals)

In [5]:
model.x = pyo.Var(model.Crop, within=pyo.NonNegativeReals) 

In [6]:
# Define the Objective Function
def obj_value_rule(model):
    return sum(model.profit[i]*model.x[i] for i in model.Crop)

model.obj = pyo.Objective(rule=obj_value_rule, sense = pyo.maximize)

In [7]:
# Land Constraint
def land_rule(model):
    return sum(1*model.x[i] for i in model.Crop)<= model.avail_land

model.land = pyo.Constraint(rule=land_rule)

In [8]:
# Labor Constraint
def labor_rule(model):
    return sum(model.w_labor[i]*model.x[i] for i in model.Crop) <= model.avail_labor

model.labor = pyo.Constraint(rule=labor_rule)

In [9]:
# Capital Constraint
def capital_rule(model):
    return sum(model.w_capital[i]*model.x[i] for i in model.Crop) <= model.avail_capital

model.capital = pyo.Constraint(rule=capital_rule)

In [10]:
# Load data
instance = model.create_instance('agriculture.dat')

In [11]:
# Solve the model through the GLPK solver
solver = pyo.SolverFactory('glpk')
solver.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 360.0, 'Upper bound': 360.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of nonzeros': 10, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03887128829956055}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
# Print the results
instance.display()
print()
print('Optimal objective value: ', pyo.value(instance.obj))

Model unknown

  Variables:
    x : Size=3, Index=Crop
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
            Corn :     0 :   6.0 :  None : False : False : NonNegativeReals
            Oats :     0 :   6.0 :  None : False : False : NonNegativeReals
        Soybeans :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 360.0

  Constraints:
    land : Size=1
        Key  : Lower : Body : Upper
        None :  None : 12.0 :  12.0
    labor : Size=1
        Key  : Lower : Body : Upper
        None :  None : 48.0 :  48.0
    capital : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 324.0 : 360.0

Optimal objective value:  360.0


In [13]:
# Print the optimal solution
print("Optimal solution:")
for crop in instance.Crop:
    print("Crop:", crop, "Amount produced:", instance.x[crop].value)

Optimal solution:
Crop: Corn Amount produced: 6.0
Crop: Soybeans Amount produced: 0.0
Crop: Oats Amount produced: 6.0


In [14]:
#Final Answer to Task 1
optimal_solution = print("Optimal solution:")
print()
for crop in instance.Crop:
    print("Crop:", crop, "Amount produced:", instance.x[crop].value, "acres")
optimal_value = pyo.value(instance.obj)

print() 
print("The Optimal Value is $", optimal_value)

Optimal solution:

Crop: Corn Amount produced: 6.0 acres
Crop: Soybeans Amount produced: 0.0 acres
Crop: Oats Amount produced: 6.0 acres

The Optimal Value is $ 360.0


# Concrete Dual Model

In [15]:
import pyomo.environ as pyo

In [16]:
# Define the index set
Crops = {'Corn','Soybeans','Oats'}

In [17]:
# Data
w_land = {'Corn':1, 'Soybeans':1, 'Oats':1}
w_labor = {'Corn':6, 'Soybeans':6, 'Oats':2}
w_capital = {'Corn':36, 'Soybeans':24, 'Oats':18}
profit = {'Corn':40, 'Soybeans':30, 'Oats':20}
avail_land = 12
avail_labor = 48
avail_capital = 360

In [18]:
# Create a Concrete model
model = pyo.ConcreteModel()

In [19]:
#Build the Dual Decision Variables
model.pi = pyo.Var({1,2,3}) 

In [20]:
#Build the Dual Objective Function
model.obj = pyo.Objective(expr = 12*model.pi[1]+48*model.pi[2] + 360*model.pi[3], sense = pyo.minimize)

In [21]:
#Define the Dual Constraints

model.con1 = pyo.Constraint(expr = model.pi[1]+6*model.pi[2] + 36*model.pi[3]  >= 40)
model.con2 = pyo.Constraint(expr = 2*model.pi[1]+6*model.pi[2] + 24*model.pi[3] >= 30)
model.con3 = pyo.Constraint(expr = model.pi[1] + 2*model.pi[2] + 18*model.pi[3] >= 20)
model.con4 = pyo.Constraint(expr = model.pi[1] >= 0)
model.con5 = pyo.Constraint(expr = model.pi[2] >= 0)
model.con6 = pyo.Constraint(expr = model.pi[3] >= 0)

In [22]:
# Solve the model through the GLPK solver
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 360.0, 'Upper bound': 360.0, 'Number of objectives': 1, 'Number of constraints': 7, 'Number of variables': 4, 'Number of nonzeros': 13, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0340578556060791}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [23]:
#Print the Model
print()
model.pprint()


1 Set Declarations
    pi_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}

1 Var Declarations
    pi : Size=3, Index=pi_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  10.0 :  None : False : False :  Reals
          2 :  None :   5.0 :  None : False : False :  Reals
          3 :  None :   0.0 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 12*pi[1] + 48*pi[2] + 360*pi[3]

6 Constraint Declarations
    con1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                       : Upper : Active
        None :  40.0 : pi[1] + 6*pi[2] + 36*pi[3] :  +Inf :   True
    con2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                         : Upper : Active
        None :  30.0 : 2*pi[1] + 6*pi[2] +

In [24]:
# Print the optimzation results
print()
model.display()  # List of all optimization results
print()
print('Optimal value: ', pyo.value(model.obj))  # Print the value of model.obj (i.e., optimal objective value)


Model unknown

  Variables:
    pi : Size=3, Index=pi_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :  None :  10.0 :  None : False : False :  Reals
          2 :  None :   5.0 :  None : False : False :  Reals
          3 :  None :   0.0 :  None : False : False :  Reals

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 360.0

  Constraints:
    con1 : Size=1
        Key  : Lower : Body : Upper
        None :  40.0 : 40.0 :  None
    con2 : Size=1
        Key  : Lower : Body : Upper
        None :  30.0 : 50.0 :  None
    con3 : Size=1
        Key  : Lower : Body : Upper
        None :  20.0 : 20.0 :  None
    con4 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 : 10.0 :  None
    con5 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 :  5.0 :  None
    con6 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 :  0.0 :  None

Optimal value:  360.0

In [25]:
# Print the optimal solution and optimal value
print(f"Optimal Solution:")
print(f"pi1 = {pyo.value(model.pi[1])}")
print(f"pi2 = {pyo.value(model.pi[2])}")
print(f"pi3 = {pyo.value(model.pi[3])}")
print(f"The Optimal Solution to the Dual Problem is ({pyo.value(model.pi[1])}, {pyo.value(model.pi[2])}, {pyo.value(model.pi[3])})")
print(f"Optimal Value of the Dual Problem = {pyo.value(model.obj)}")

Optimal Solution:
pi1 = 10.0
pi2 = 5.0
pi3 = 0.0
The Optimal Solution to the Dual Problem is (10.0, 5.0, 0.0)
Optimal Value of the Dual Problem = 360.0
